In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import math

In [2]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        if snode == 80:
            continue
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    G.remove_node(80)
    return

In [3]:
def init_graph_attr(G,AdjFile,df,col1, col2):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["KGISTalukN"].tolist()
    node_list.insert(80, "")
    nodeAttr = {}
    init_graph(G,node_adj_frame)
    capability_vector = list(zip(df[col1], df[col2]))
    node_attri_dict = dict(zip(df["Taluka"],capability_vector))
    node_attri_dict = dict((k.lower(), v) for k, v in node_attri_dict.items())
    for i in range(len(node_adj_frame)):
        temp = {}
        if i == 80:
            continue
        temp["capabilityvector"] = node_attri_dict[node_list[i].lower()]
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp
    nt = {}
    nt["capabilityvector"] = node_attri_dict[node_list[226].lower()]
    nt["nodeStress"] = 0
    nt["name"] = "Hadagali"
    nodeAttr[226] = nt
    nx.set_node_attributes(G, nodeAttr)

In [4]:
G = nx.Graph()
df = pd.read_excel('../input_files/Combined_PIA_IMR_MMR.xlsx')

In [5]:
# df.iloc[7]
dist_taluka_dict = defaultdict(list)
for k, v in zip(df["District_GIS"], df["Taluka"]):
    dist_taluka_dict[k].append(v)

In [6]:
def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1
def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1
def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

In [7]:
def get_node_stress(G):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0,0]
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"].lower()]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [8]:
def get_node_stability(G):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0,0]
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"].lower()]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [9]:
df['Taluka'] = df['Taluka'].apply(str.lower)
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized MMR", "Normalized IMR")
initialstress = get_node_stress(G)

In [10]:
df["Initial Stress"] = df["Taluka"].map(initialstress)

In [11]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC-20%)", "Normalized MMR (ANC -20%)")
ANCminus20stress = get_node_stress(G)
df["Stress(ANC - 20%)"] = df["Taluka"].map(ANCminus20stress)

In [12]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC+20%)", "Normalized MMR (ANC +20%)")
ANCplus20stress = get_node_stress(G)
df["Stress(ANC + 20%)"] = df["Taluka"].map(ANCplus20stress)

In [13]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC-10%)", "Normalized MMR (ANC -10%)")
ANCminus10stress = get_node_stress(G)
df["Stress(ANC - 10%)"] = df["Taluka"].map(ANCminus10stress)

In [14]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC+10%)", "Normalized MMR (ANC +10%)")
ANCplus10stress = get_node_stress(G)
df["Stress(ANC + 10%)"] = df["Taluka"].map(ANCplus10stress)

In [15]:
def aggregate(taluka_dict):
    dist_stress = {}
    for dist, taluks in dist_taluka_dict.items():
        agg_stress = 0
        for taluk in taluks:
            try:
                agg_stress = agg_stress + taluka_dict[taluk.lower()]
            except(KeyError):
                pass
        dist_stress[dist] = agg_stress/len(taluks)
    return dist_stress

In [16]:
result = aggregate(get_node_stress(G))
# result : it has district vs stress

In [17]:
df_res = pd.DataFrame.from_dict(result, orient='index')
df_res.reset_index(inplace=True)
df_res.columns = ['Distirct', 'Stress ANC-20%']

In [18]:
# df_res.to_excel('../windows_2d/2D_Aggregate_Stability.xlsx', index=False)

In [19]:
df_imp_IMR = pd.read_excel("../input_files/PIA_Normalized_IMR.xlsx")
df_imp_MMR = pd.read_excel("../input_files/PIA_Normalized_MMR.xlsx")

In [20]:
combined_Impact_ANC = pd.DataFrame()

In [21]:
aggregate_df = pd.DataFrame()

In [22]:
# ===================================================FOR ANC -20%==============================================================
# Getting the graph ready for ANC -20%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC-20%)", "Normalized MMR (ANC -20%)")

# These dict have the names vs Impact Score for ANC -20%
IMR_impact_dict = dict(zip(df_imp_IMR["Taluka"],df_imp_IMR["Impact Score (ANC -20%)"]))
MMR_impact_dict = dict(zip(df_imp_MMR["Taluka"],df_imp_MMR["Impact Score (ANC -20%)"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (ANC - 20%)',1:'IMPACT_SCORE_MMR (ANC - 20%)', 2:'STRESS_SCORE (ANC - 20%)'}, inplace = True)
aggregate_df = aI_df

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (ANC - 20%)', 1:'STRESS_SCORE (ANC - 20%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (ANC - 20%)', 1:'STRESS_SCORE (ANC - 20%)'}, inplace = True)

In [23]:
combined_Impact_ANC = trdIMR
combined_Impact_ANC['IMPACT_SCORE_MMR (ANC - 20%)'] = trdMMR['IMPACT_SCORE_MMR (ANC - 20%)']
# combined_Impact_ANC['']

In [24]:
# ===================================================FOR ANC -10%==============================================================
# Getting the graph ready for ANC -10%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC-10%)", "Normalized MMR (ANC -10%)")

# These dict have the names vs Impact Score for ANC -10%
IMR_impact_dict = dict(zip(df_imp_IMR["Taluka"],df_imp_IMR["Impact Score (ANC -10%)"]))
MMR_impact_dict = dict(zip(df_imp_MMR["Taluka"],df_imp_MMR["Impact_____Deprivation % - Households with with any woman has not received at least 4 antenatal care visits for the most recent birth or has not received assistance from trained skilled medical personnel during the most recent childbirth._____-10%"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (ANC - 10%)',1:'IMPACT_SCORE_MMR (ANC - 10%)', 2:'STRESS_SCORE (ANC -10%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (ANC - 10%)'] = aI_df['IMPACT_SCORE_IMR (ANC - 10%)']
aggregate_df['IMPACT_SCORE_MMR (ANC - 10%)'] = aI_df['IMPACT_SCORE_MMR (ANC - 10%)']
aggregate_df['STRESS_SCORE (ANC -10%)'] = aI_df['STRESS_SCORE (ANC -10%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (ANC - 10%)', 1:'STRESS_SCORE (ANC -10%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (ANC - 10%)', 1:'STRESS_SCORE (ANC -10%)'}, inplace = True)

In [25]:
# trdIMR[""]
combined_Impact_ANC['IMPACT_SCORE_MMR (ANC - 10%)'] = trdMMR['IMPACT_SCORE_MMR (ANC - 10%)']
combined_Impact_ANC['IMPACT_SCORE_IMR (ANC - 10%)'] = trdIMR['IMPACT_SCORE_IMR (ANC - 10%)']
combined_Impact_ANC['STRESS_SCORE (ANC - 10%)'] = trdMMR['STRESS_SCORE (ANC -10%)']
combined_Impact_ANC

,IMPACT_SCORE_IMR (ANC - 20%),STRESS_SCORE (ANC - 20%),IMPACT_SCORE_MMR (ANC - 20%),IMPACT_SCORE_MMR (ANC - 10%),IMPACT_SCORE_IMR (ANC - 10%),STRESS_SCORE (ANC - 10%)
afzalpur,0.490016,0.113143,0.411532,0.411532,0.490016,0.114822
ajjampura,0.387197,0.253022,0.053511,0.053511,0.387197,0.253273
aland,0.689810,0.111469,0.291245,0.291245,0.689810,0.110768
alnavara,0.368436,0.048910,0.157936,0.157936,0.368436,0.048998
alur,0.261613,0.184957,0.160139,0.160139,0.261613,0.185602
...,...,...,...,...,...,...
yadrami,0.731272,0.237260,0.389470,0.389470,0.731272,0.239885
yalandur,0.353070,0.401624,0.149711,0.149711,0.353070,0.402082
yelahanka,0.064048,0.304602,0.205265,0.205265,0.064048,0.306465
yelburga,0.886327,0.212719,0.586761,0.586761,0.886327,0.211859


In [26]:
# ===================================================FOR ANC +10%==============================================================
# Getting the graph ready for ANC +10%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC+10%)", "Normalized MMR (ANC +10%)")

# These dict have the names vs Impact Score for ANC +10%
IMR_impact_dict = dict(zip(df_imp_IMR["Taluka"],df_imp_IMR["Impact Score (ANC +10%)"]))
MMR_impact_dict = dict(zip(df_imp_MMR["Taluka"],df_imp_MMR["Impact_____Deprivation % - Households with with any woman has not received at least 4 antenatal care visits for the most recent birth or has not received assistance from trained skilled medical personnel during the most recent childbirth._____+10%"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (ANC + 10%)',1:'IMPACT_SCORE_MMR (ANC + 10%)', 2:'STRESS_SCORE (ANC +10%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (ANC + 10%)'] = aI_df['IMPACT_SCORE_IMR (ANC + 10%)']
aggregate_df['IMPACT_SCORE_MMR (ANC + 10%)'] = aI_df['IMPACT_SCORE_MMR (ANC + 10%)']
aggregate_df['STRESS_SCORE (ANC +10%)'] = aI_df['STRESS_SCORE (ANC +10%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (ANC + 10%)', 1:'STRESS_SCORE (ANC +10%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (ANC + 10%)', 1:'STRESS_SCORE (ANC +10%)'}, inplace = True)

In [27]:
combined_Impact_ANC['IMPACT_SCORE_MMR (ANC + 10%)'] = trdMMR['IMPACT_SCORE_MMR (ANC + 10%)']
combined_Impact_ANC['IMPACT_SCORE_IMR (ANC + 10%)'] = trdIMR['IMPACT_SCORE_IMR (ANC + 10%)']
combined_Impact_ANC['STRESS_SCORE (ANC + 10%)'] = trdMMR['STRESS_SCORE (ANC +10%)']
combined_Impact_ANC

,IMPACT_SCORE_IMR (ANC - 20%),STRESS_SCORE (ANC - 20%),IMPACT_SCORE_MMR (ANC - 20%),IMPACT_SCORE_MMR (ANC - 10%),IMPACT_SCORE_IMR (ANC - 10%),STRESS_SCORE (ANC - 10%),IMPACT_SCORE_MMR (ANC + 10%),IMPACT_SCORE_IMR (ANC + 10%),STRESS_SCORE (ANC + 10%)
afzalpur,0.490016,0.113143,0.411532,0.411532,0.490016,0.114822,0.588468,0.509984,0.117308
ajjampura,0.387197,0.253022,0.053511,0.053511,0.387197,0.253273,0.946489,0.612803,0.253129
aland,0.689810,0.111469,0.291245,0.291245,0.689810,0.110768,0.708755,0.310190,0.108709
alnavara,0.368436,0.048910,0.157936,0.157936,0.368436,0.048998,0.842064,0.631564,0.049114
alur,0.261613,0.184957,0.160139,0.160139,0.261613,0.185602,0.839861,0.738387,0.185792
...,...,...,...,...,...,...,...,...,...
yadrami,0.731272,0.237260,0.389470,0.389470,0.731272,0.239885,0.610530,0.268728,0.244188
yalandur,0.353070,0.401624,0.149711,0.149711,0.353070,0.402082,0.850289,0.646930,0.399138
yelahanka,0.064048,0.304602,0.205265,0.205265,0.064048,0.306465,0.794735,0.935952,0.307630
yelburga,0.886327,0.212719,0.586761,0.586761,0.886327,0.211859,0.413239,0.113673,0.208895


In [28]:
# ===================================================FOR ANC +20%==============================================================
# Getting the graph ready for ANC -10%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (ANC+20%)", "Normalized MMR (ANC +20%)")

# These dict have the names vs Impact Score for ANC +20%
IMR_impact_dict = dict(zip(df_imp_IMR["Taluka"],df_imp_IMR["Impact Score (ANC +20%)"]))
MMR_impact_dict = dict(zip(df_imp_MMR["Taluka"],df_imp_MMR["Impact_____Deprivation % - Households with with any woman has not received at least 4 antenatal care visits for the most recent birth or has not received assistance from trained skilled medical personnel during the most recent childbirth._____+20%"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (ANC + 20%)',1:'IMPACT_SCORE_MMR (ANC + 20%)', 2:'STRESS_SCORE (ANC +20%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (ANC + 20%)'] = aI_df['IMPACT_SCORE_IMR (ANC + 20%)']
aggregate_df['IMPACT_SCORE_MMR (ANC + 20%)'] = aI_df['IMPACT_SCORE_MMR (ANC + 20%)']
aggregate_df['STRESS_SCORE (ANC +20%)'] = aI_df['STRESS_SCORE (ANC +20%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (ANC + 20%)', 1:'STRESS_SCORE (ANC +20%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (ANC + 20%)', 1:'STRESS_SCORE (ANC +20%)'}, inplace = True)

In [29]:
combined_Impact_ANC['IMPACT_SCORE_MMR (ANC + 20%)'] = trdMMR['IMPACT_SCORE_MMR (ANC + 20%)']
combined_Impact_ANC['IMPACT_SCORE_IMR (ANC + 20%)'] = trdIMR['IMPACT_SCORE_IMR (ANC + 20%)']
combined_Impact_ANC['STRESS_SCORE (ANC + 20%)'] = trdMMR['STRESS_SCORE (ANC +20%)']
combined_Impact_ANC

,IMPACT_SCORE_IMR (ANC - 20%),STRESS_SCORE (ANC - 20%),IMPACT_SCORE_MMR (ANC - 20%),IMPACT_SCORE_MMR (ANC - 10%),IMPACT_SCORE_IMR (ANC - 10%),STRESS_SCORE (ANC - 10%),IMPACT_SCORE_MMR (ANC + 10%),IMPACT_SCORE_IMR (ANC + 10%),STRESS_SCORE (ANC + 10%),IMPACT_SCORE_MMR (ANC + 20%),IMPACT_SCORE_IMR (ANC + 20%),STRESS_SCORE (ANC + 20%)
afzalpur,0.490016,0.113143,0.411532,0.411532,0.490016,0.114822,0.588468,0.509984,0.117308,0.588468,0.509984,0.118335
ajjampura,0.387197,0.253022,0.053511,0.053511,0.387197,0.253273,0.946489,0.612803,0.253129,0.946489,0.612803,0.252907
aland,0.689810,0.111469,0.291245,0.291245,0.689810,0.110768,0.708755,0.310190,0.108709,0.708755,0.310190,0.107542
alnavara,0.368436,0.048910,0.157936,0.157936,0.368436,0.048998,0.842064,0.631564,0.049114,0.842064,0.631564,0.049157
alur,0.261613,0.184957,0.160139,0.160139,0.261613,0.185602,0.839861,0.738387,0.185792,0.839861,0.738387,0.185626
...,...,...,...,...,...,...,...,...,...,...,...,...
yadrami,0.731272,0.237260,0.389470,0.389470,0.731272,0.239885,0.610530,0.268728,0.244188,0.610530,0.268728,0.246105
yalandur,0.353070,0.401624,0.149711,0.149711,0.353070,0.402082,0.850289,0.646930,0.399138,0.850289,0.646930,0.396770
yelahanka,0.064048,0.304602,0.205265,0.205265,0.064048,0.306465,0.794735,0.935952,0.307630,0.794735,0.935952,0.307605
yelburga,0.886327,0.212719,0.586761,0.586761,0.886327,0.211859,0.413239,0.113673,0.208895,0.413239,0.113673,0.207142


In [30]:
# Now, we have a df(combined_Impact_ANC), which has Stability and 2D impacts for Talukas. We have to aggregate them now.

In [31]:
combined_Impact_ANC.to_excel('../windows_2d/2D_stress_impact_talukas.xlsx')

In [32]:
# adf = aggregate_df.rename(columns={'STRESS_SCORE (ANC -20%)': 'STABILITY_SCORE (ANC -20%)', 'STRESS_SCORE (ANC -10%)' : 'STABILITY_SCORE (ANC -10%)', 'STRESS_SCORE (ANC +10%)' : 'STABILITY_SCORE (ANC +10%)', 'STRESS_SCORE (ANC +20%)' : 'STABILITY_SCORE (ANC +20%)'})

In [33]:
# aggregate_df = adf

In [34]:
aggregate_df

,IMPACT_SCORE_IMR (ANC - 20%),IMPACT_SCORE_MMR (ANC - 20%),STRESS_SCORE (ANC - 20%),IMPACT_SCORE_IMR (ANC - 10%),IMPACT_SCORE_MMR (ANC - 10%),STRESS_SCORE (ANC -10%),IMPACT_SCORE_IMR (ANC + 10%),IMPACT_SCORE_MMR (ANC + 10%),STRESS_SCORE (ANC +10%),IMPACT_SCORE_IMR (ANC + 20%),IMPACT_SCORE_MMR (ANC + 20%),STRESS_SCORE (ANC +20%)
Kalburgi,0.554321,0.410204,0.207544,0.554321,0.410204,0.208459,0.445679,0.589796,0.209839,0.445679,0.589796,0.210427
Chikkamagaluru,0.265681,0.159505,0.201318,0.265681,0.159505,0.202587,0.734319,0.840495,0.204482,0.734319,0.840495,0.205277
Dharwad,0.228346,0.195689,0.274857,0.228346,0.195689,0.275962,0.771654,0.804311,0.277259,0.771654,0.804311,0.277692
Hassan,0.305620,0.159257,0.327954,0.305620,0.159257,0.329572,0.694380,0.840743,0.330974,0.694380,0.840743,0.331253
Bengaluru_Urban,0.071142,0.112375,0.187943,0.071142,0.112375,0.188947,0.928858,0.887625,0.189402,0.928858,0.887625,0.189263
Uttara_Kannada,0.634912,0.299434,0.211438,0.634912,0.299434,0.211530,0.365088,0.700566,0.210598,0.365088,0.700566,0.209933
Belagavi,0.517916,0.351575,0.174305,0.517916,0.351575,0.174338,0.482084,0.648425,0.173768,0.482084,0.648425,0.173350
Bidar,0.597858,0.461555,0.136331,0.597858,0.461555,0.135412,0.402142,0.538445,0.132665,0.402142,0.538445,0.131100
Vijayapura,0.585366,0.642162,0.146893,0.585366,0.642162,0.147429,0.414634,0.357838,0.148065,0.414634,0.357838,0.148286
Bagalkot,0.366161,0.269064,0.184659,0.366161,0.269064,0.185518,0.633839,0.730936,0.186169,0.633839,0.730936,0.186262


In [35]:
aggregate_df.to_excel('../windows_2d/2D_aggregate_stress.xlsx')

In [36]:
# ==========================================================================SCORE CALCULATIONS======================================================================================

In [37]:
df_score = pd.DataFrame()
df_score['Taluka'] = df['Taluka']

In [38]:
def dissonance_score(df, score_list, i):
    for j in range(len(df)):
        str_imr = 'IMPACT_SCORE_IMR (ANC ' + i + '0%)'
        str_mmr = 'IMPACT_SCORE_MMR (ANC ' + i + '0%)'
        str_stress = 'STRESS_SCORE (ANC ' + i + '0%)'
        mean_impact = (df[str_imr][j] + df[str_mmr][j]) / 2
        dissonance = abs(df[str_imr][j] - df[str_mmr][j])
        score = (mean_impact * (1 - df[str_stress][j])) / dissonance
        score_list.append(score)

In [39]:
temp = ['+ 2', '+ 1', '- 1', '- 2']
for i in temp:
    score_list = []
    dissonance_score(combined_Impact_ANC, score_list, i)
    str = 'Score | ANC ' + i + '0%'
    df_score[str] = score_list

In [40]:
df_score.to_excel('../windows_2d/2D_taluka_scores.xlsx')

In [41]:
# for i in range(len(df_score)):
#     print('Taluka name: ', df_score['Taluka'][i])
#     print('Score -20: ', df_score['Score | ANC - 20%'][i])
#     print('======================================')

In [42]:
aggregated_score_df = pd.DataFrame()
aggregated_score_df['District'] = dist_taluka_dict.keys()
# aggregated_score_df

In [43]:
for i in temp:
    temp_dict = dict(zip(df_score['Taluka'],df_score['Score | ANC ' + i + '0%']))
    agg_dict = aggregate(temp_dict)
    aggregated_score_df['Score | ANC ' + i + '0%'] = agg_dict.values()

In [44]:
aggregated_score_df.to_excel('../windows_2d/2D_district_scores.xlsx')